# Welcome to the Tutorial of python plot

In lesson two, we would like to plot some UV-vis spectra data.

From absorption vs wavelength tranfer to Tauc plot.

After normal plot, we will define a linaer fit function for Band gap estimation.



In [ ]:
# for colab env, ipympl for %matplotlib widget work, and lmfit for fitting.
!pip install ipympl lmfit
get_ipython().kernel.do_shutdown(restart=True)

In [ ]:
# activate "Third-party Jupyter widgets" functios.
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Condition 1 
# Read data from public github.

addres_head = f'https://raw.githubusercontent.com/VolHC/Lab_plot_tutorial/refs/heads/main/Basic_plot'

fn = [f'{addres_head}/demo_data/UV_demo-1.txt',
      f'{addres_head}/demo_data/UV_demo-2.txt',
      f'{addres_head}/demo_data/UV_demo-3.txt',
      f'{addres_head}/demo_data/UV_demo-4.txt',]


In [ ]:
# Condition 2
# Read data from local ascii type data.

path = f'/Users/chen.bh/git/Lab_plot_tutorial/Basic_plot'

fn = [f'{path}/demo_data/UV_demo-1.txt',
      f'{path}/demo_data/UV_demo-2.txt',
      f'{path}/demo_data/UV_demo-3.txt',
      f'{path}/demo_data/UV_demo-4.txt',]

fn

In [ ]:
# Condition 3
# Using colab and read data from local ascii type data.

from google.colab import files
uploaded = files.upload()
uploaded.keys()

fn  = list(uploaded.keys())
fn

In [ ]:
# use "for loop" do multiple plot.

uv = [(np.loadtxt(n, skiprows=1)[::-1].T)
      for n in fn]

x = [ar[0]
     for ar in uv]
y = [ar[1]
     for ar in uv]

colors = ['k', 'b', 'r', 'g']
lines  = ['-']*4
symbol = ['']*4
labels = ['Demo1', 'Demo2', 'Demo3', 'Demo4']

for i in range(4):
    plt.plot(x[i], y[i], color=colors[i], marker=symbol[i], linestyle=lines[i], label=labels[i])

plt.title("UV-Vis spectra")
plt.xlabel('Wavelength (nm)')
plt.ylabel('Absorbance')
plt.legend()
plt.show()

# From UV–Vis Absorbance to Tauc Plot

## Step 1 — Photon energy (eV)
Convert wavelength (nm) to photon energy:

`E = h * c / λ`

where  

- `E` = photon energy (J)  
- `h = 6.62607015 × 10⁻³⁴ J·s` (Planck constant)  
- `c = 2.99792458 × 10⁸ m/s` (speed of light in vacuum)  
- `λ` = wavelength (m)

### Convert to electronvolts (eV)

Since `1 eV = 1.602176634 × 10⁻¹⁹ J`:

`E (eV) = (h * c / λ) / e`

= `(6.62607015e-34 * 2.99792458e8) / (λ * 1.602176634e-19)`

Simplify constants:

`E (eV) ≈ 1239.841984 / λ (nm)`

Example: for `λ = 500 nm` → `hν = 1239.841984 / 500 = 2.479 eV`

---

## Step 2 — Absorption coefficient (α)
From absorbance `A` (base-10) and path length `d` (cm):

`α = ln(10) * A / d ≈ 2.303 * A / d    [cm^-1]`

---

## Step 3 — Tauc quantity
Compute the Tauc y-value `y = (α * hν)^p` and plot `y` vs `hν`.

- **Direct allowed transition**
  - `p = 2`  →  `y = (α * hν)^2`

- **Indirect allowed transition**
  - `p = 1/2` → `y = (α * hν)^(1/2)`

---

## Step 4 — Estimating the bandgap (Eg)
1. Identify the (quasi-)linear portion of the Tauc plot.  
2. Fit a straight line `y = m * (hν) + b` to that region.  
3. Extrapolate to `y = 0`: the bandgap is

`Eg = -b / m`   (in eV)

---

## Quick checklist
- Ensure `λ` in nm and `d` in cm.  
- Use `α = 2.303*A/d`.  
- Try both exponents (`p = 2` and `p = 0.5`) if transition type is unknown.  
- Subtract baseline if absorbance has an offset (instrument background).



In [ ]:
def absorbance_to_tauc(wavelength_nm, absorbance, path_length_cm=1.0, transition='direct'):
    """
    Convert UV–Vis absorbance data to Tauc plot quantities.

    Parameters
    ----------
    wavelength_nm : 1D array-like
        Wavelengths in nm.
    absorbance : 1D array-like
        Absorbance (log10 I0/I).
    path_length_cm : float, default 1.0
        Optical path length in cm.
    transition : str, 'direct' or 'indirect'
        Transition type for Tauc plot.
        'direct'   -> exponent = 2
        'indirect' -> exponent = 0.5

    Returns
    -------
    hv : ndarray
        Photon energy in eV.
    alpha : ndarray
        Absorption coefficient in cm^-1.
    y_tauc : ndarray
        Tauc quantity (alpha*hv)^p, where p=2 or 0.5.
    """
    wl = np.asarray(wavelength_nm).flatten()
    A = np.asarray(absorbance).flatten()
    if wl.size != A.size:
        raise ValueError("wavelength and absorbance arrays must have same length")

    # Photon energy (eV)
    
    hv = 1239.841984 / wl

    # Absorption coefficient (cm^-1)
    alpha = 2.303 * A / float(path_length_cm)

    # Exponent selection
    exponent = 2.0 if transition.lower().startswith('d') else 0.5
    # print(exponent)

    # Tauc y quantity
    y_tauc = (alpha * hv) ** exponent

    return hv, alpha, y_tauc, 


In [ ]:
# convert Absorbence to Tauc first
# %matplotlib notebook
%matplotlib widget
plt.close()
tauc_x = [absorbance_to_tauc(ar[0], ar[1])[0]
          for ar in uv]

tauc_y = [absorbance_to_tauc(ar[0], ar[1])[2]
          for ar in uv]

colors = ['k', 'b', 'r', 'g']
lines  = ['-']*4
symbol = ['']*4
labels = ['Demo1', 'Demo2', 'Demo3', 'Demo4']

for i in range(4):
    plt.plot(tauc_x[i], tauc_y[i], color=colors[i], marker=symbol[i], linestyle=lines[i], label=labels[i])

plt.title("Tauc plot (direct band gap)")
plt.xlabel('Energy (eV)')
plt.ylabel(r"$(\alpha h\nu)^2$")
plt.xlim(3,6.25)
plt.ylim(-0.1, 5200)
plt.legend()
plt.show()

In [ ]:
# define a linear model with own parameter name.

from lmfit import Model

# y = m * (hν) + b
def tauc_linear(e, m, b):
    return(m*e+b)

tl_model = Model(tauc_linear)

## Error Propagation Formulas

Given two independent quantities:

$$
A \pm \sigma_A, \quad B \pm \sigma_B
$$

---

### Ratio

$$
R = \frac{A}{B}, \quad 
\sigma_R = R \,\sqrt{\left(\frac{\sigma_A}{A}\right)^2 + \left(\frac{\sigma_B}{B}\right)^2}
$$

---

### Product

$$
P = A \times B, \quad 
\sigma_P = |P| \,\sqrt{\left(\frac{\sigma_A}{A}\right)^2 + \left(\frac{\sigma_B}{B}\right)^2}
$$

---

### Sum and Difference

$$
S = A + B, \quad D = A - B
$$

$$
\sigma_S = \sigma_D = \sqrt{\sigma_A^2 + \sigma_B^2}
$$


In [ ]:
def ratio_with_error(A, sigma_A, B, sigma_B):
    R = A / B
    sigma_R = abs(R) * np.sqrt((sigma_A / A)**2 + (sigma_B / B)**2)
    return np.round(R, 5), np.round(sigma_R, 5)

In [ ]:
%matplotlib widget
plt.close()

# give a try of one UV-vis plot.
# giving low and high limit of hν.

def fit_plot(e, tauc, low_e, high_e, plot_fit=False, plot_label=None, plot_line_color=None):
    idx = np.where((e >= low_e) & (e <= high_e))[0]
    
    result = tl_model.fit(tauc[idx], e=e[idx], m=-1, b=-10)
    print(result.fit_report())
    
    # Band Gap calculation
    m, s_m = result.params['m'].value, result.params['m'].stderr
    b, s_b = result.params['b'].value, result.params['b'].stderr
    gap, s_gap = ratio_with_error(b*-1, s_b, m, s_m)
    
    print(f'Band gap : {gap}({s_gap})')
    
    if plot_fit:
		# create fit line
        fit_e = np.linspace(gap, e[idx[0]], 1000)
        fit_tauc = np.poly1d([m, b])(fit_e) # f(x), f = np.poly1d([m, b]), x = fit_e
        # print(fit_e, fit_tauc)

        # plot single UV and fit result
        pcl = plot_line_color or 'k'
        pl  = plot_label or 'Demo'
        plt.plot(e, tauc, '-', color=pcl, label=pl)
        plt.plot(fit_e, fit_tauc, '--', color=pcl)
        plt.title("Tauc plot (direct band gap)")
        plt.xlabel('Energy (eV)')
        plt.ylabel(r"$(\alpha h\nu)^2$")
        # plt.xlim(3,6.25)     # adjust x axis range
        # plt.ylim(-0.1, 5200) # adjust y axis range
        plt.legend()
plt.show(block=False)


# useage
fit_plot(tauc_x[0], tauc_y[0], 5.2, 5.3, plot_fit=True)
fit_plot(tauc_x[1], tauc_y[1], 4.3, 4.5, plot_fit=True, plot_line_color='b')


    